In [1]:
# ===============================
# Connect Four Simulation Notebook
# ===============================

# --- Imports ---
import numpy as np
import pandas as pd
from itertools import combinations

# Import functions from SRC
from SRC.board import create_board, make_move, check_win
from SRC.player import random_player, heuristic_player, intelligent_player
from SRC.simulation import simulate_game_with_wins

# --- Parameters ---
NUM_COLS = 7
BASE_SIM = 100  # default number of simulations per pairing

# Define players
players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b, p: intelligent_player(b, p, depth=1),
    "11 Years": lambda b, p: intelligent_player(b, p, depth=2)
}

# Optional: automatic simulation count per pairing (between 100-500)
def determine_sim_count(balance_estimate):
    """Return simulation count based on estimated balance"""
    return int(np.clip(BASE_SIM * (1 + balance_estimate * 2), 100, 500))

# Estimated balance between players (0 = unbalanced, 1 = very balanced)
balance_estimates = {
    ("4 Years","7 Years"): 0.1,
    ("4 Years","9 Years"): 0.3,
    ("4 Years","11 Years"): 0.05,
    ("7 Years","9 Years"): 0.5,
    ("7 Years","11 Years"): 0.7,
    ("9 Years","11 Years"): 0.8
}

# --- Run all simulations ---
results = []

for p1_name, p2_name in combinations(players.keys(), 2):

    # Number of simulations for this pairing
    sim_count = determine_sim_count(balance_estimates.get((p1_name, p2_name), 0.3))

    # Initialize counts
    outcomes = {p1_name:0, p2_name:0, "Draw":0}

    # Run simulations
    for _ in range(sim_count):
        winner = simulate_game_with_wins(players[p1_name], players[p2_name],
                                        moves_count_p1=None,
                                        moves_count_p2=None,
                                        wins_count_p1=None,
                                        wins_count_p2=None)
        if winner == 1:
            outcomes[p1_name] += 1
        elif winner == 2:
            outcomes[p2_name] += 1
        else:
            outcomes["Draw"] += 1

    # Calculate probabilities
    prob_p1 = outcomes[p1_name] / sim_count
    prob_p2 = outcomes[p2_name] / sim_count
    prob_draw = outcomes["Draw"] / sim_count

    # Standard errors
    se_p1 = np.sqrt(prob_p1 * (1 - prob_p1) / sim_count)
    se_p2 = np.sqrt(prob_p2 * (1 - prob_p2) / sim_count)
    se_draw = np.sqrt(prob_draw * (1 - prob_draw) / sim_count)

    # Betting odds (simple)
    quote_p1 = 1 / prob_p1 if prob_p1 > 0 else np.inf
    quote_p2 = 1 / prob_p2 if prob_p2 > 0 else np.inf
    quote_draw = 1 / prob_draw if prob_draw > 0 else np.inf

    # Append to results
    results.append({
        "Player 1": p1_name,
        "Player 2": p2_name,
        "Simulations": sim_count,
        "P1 Wins (%)": prob_p1*100,
        "P1 SE (%)": se_p1*100,
        "P1 Quote": quote_p1,
        "P2 Wins (%)": prob_p2*100,
        "P2 SE (%)": se_p2*100,
        "P2 Quote": quote_p2,
        "Draw (%)": prob_draw*100,
        "Draw SE (%)": se_draw*100,
        "Draw Quote": quote_draw
    })

# --- Convert results to DataFrame ---
df_results = pd.DataFrame(results)

# Display nicely
pd.set_option('display.float_format', lambda x: '%.1f' % x)
df_results


ModuleNotFoundError: No module named 'pandas'